## Get Threshold

In [ ]:
import  os
import  SimpleITK  as  sitk

In [21]:
import  os
import  SimpleITK  as  sitk

ct_path = 'E:\skripsi\Task02_Heart\imagesTr'
seg_path = 'E:\skripsi\Task02_Heart\labelsTr'

# at the threshold
upper  =  1400
lower  =  -1400

num_point  =  0.0   # number of voxels belonging to the organ
num_inlier  =  0.0   # The number of voxels whose gray value is within the threshold in the organ voxel

for  ct_file  in  os.listdir(ct_path):

    # Read the original ct data and the corresponding gold standard into the memory
    ct  =  sitk.ReadImage(os.path.join(ct_path , ct_file), sitk.sitkInt16 )
    ct_array  =  sitk.GetArrayFromImage(ct)

    seg  =  sitk.ReadImage(os.path.join(seg_path , ct_file), sitk.sitkInt8 )
    seg_array  =  sitk.GetArrayFromImage(seg)

    # Select the voxels that belong to the organ
    organ_roi  =  ct_array[seg_array  >  0]

    inliers  = (( organ_roi  <=  upper ) * ( organ_roi  >=  lower )).astype(int).sum()

    print ('{:.4}%'.format(inliers/organ_roi.shape[0] * 100))
    print ('------------')

    num_point  += organ_roi.shape[0]
    num_inlier  +=  inliers

print ( num_inlier  /  num_point )

97.31%
------------
99.66%
------------
99.43%
------------
99.1%
------------
97.35%
------------
99.62%
------------
99.16%
------------
99.07%
------------
100.0%
------------
96.5%
------------
99.11%
------------
99.97%
------------
99.51%
------------
99.44%
------------
99.14%
------------
95.75%
------------
95.83%
------------
99.24%
------------
83.03%
------------
92.75%
------------
0.9785147831852965


## GetSliceWithSegment after thresholding

In [1]:
# https://github.com/assassint2017/abdominal-multi-organ-segmentation/blob/master/data_perpare/get_data.py
import os
import shutil
from time import time

import numpy as np
import SimpleITK as sitk
import scipy.ndimage as ndimage

In [2]:
new_ct_path = 'E:\skripsi\Dataset\Train\CT'
new_seg_path = 'E:\skripsi\Dataset\Train\GT'

In [3]:
if os.path.exists('E:\skripsi\Dataset\Train') is True:
    shutil.rmtree('E:\skripsi\Dataset\Train')
os.mkdir('E:\skripsi\Dataset\Train')
os.mkdir(new_ct_path)
os.mkdir(new_seg_path)

In [6]:
upper = 350
lower = -upper
slice_thickness = 3
down_scale = 0.5
expand_slice = 10

file_index = 0

In [22]:
ct_path = 'E:\skripsi\Task02_Heart\imagesTr'
seg_path = 'E:\skripsi\Task02_Heart\labelsTr'
upper = 1500
lower = -1500
file_index = 0

start_time = time()

for ct_file in os.listdir(ct_path):

    ct = sitk.ReadImage(os.path.join(ct_path, ct_file), sitk.sitkInt16)
    ct_array = sitk.GetArrayFromImage(ct)

    seg = sitk.ReadImage(os.path.join(seg_path, ct_file), sitk.sitkInt8)
    seg_array = sitk.GetArrayFromImage(seg)

    # 对CT和金标准使用插值算法进行插值来统一轴向的spacing，插值之后的array依然是int类型
    ct_array = ndimage.zoom(ct_array, (ct.GetSpacing()[-1] / slice_thickness, down_scale, down_scale), order=3)

    # 对金标准插值不应该使用高级插值方式，这样会破坏边界部分，总之这次错误也说明了检查数据的重要性
    seg_array = ndimage.zoom(seg_array, (ct.GetSpacing()[-1] / slice_thickness, down_scale, down_scale), order=0)

    # 将灰度值在阈值之外的截断掉
    ct_array[ct_array > upper] = upper
    ct_array[ct_array < lower] = lower

    # 找到包含器官的slice
    z = np.any(seg_array, axis=(1, 2))
    start_slice, end_slice = np.where(z)[0][[0, -1]]

    # 两个方向上各扩张slice
    if start_slice - expand_slice < 0:
        start_slice = 0
    else:
        start_slice -= expand_slice

    if end_slice + expand_slice >= seg_array.shape[0]:
        end_slice = seg_array.shape[0] - 1
    else:
        end_slice += expand_slice

    ct_array = ct_array[start_slice:end_slice + 1, :, :]
    seg_array = seg_array[start_slice:end_slice + 1, :, :]

    print('file name:', ct_file)
    print('shape:', ct_array.shape)

    # 保存数据
    new_ct_array = ct_array
    new_seg_array = seg_array

    new_ct = sitk.GetImageFromArray(new_ct_array)

    new_ct.SetDirection(ct.GetDirection())
    new_ct.SetOrigin(ct.GetOrigin())
    new_ct.SetSpacing(
        (ct.GetSpacing()[0] * int(1 / down_scale), ct.GetSpacing()[1] * int(1 / down_scale), slice_thickness))

    new_seg = sitk.GetImageFromArray(new_seg_array)

    new_seg.SetDirection(ct.GetDirection())
    new_seg.SetOrigin(ct.GetOrigin())
    new_seg.SetSpacing(
        (ct.GetSpacing()[0] * int(1 / down_scale), ct.GetSpacing()[1] * int(1 / down_scale), slice_thickness))

#     new_ct_name = 'img-' + str(file_index) + '.nii'
#     new_seg_name = 'label-' + str(file_index) + '.nii'

#     sitk.WriteImage(new_ct, os.path.join(new_ct_path, new_ct_name))
#     sitk.WriteImage(new_seg, os.path.join(new_seg_path, new_seg_name))

    # 每处理完一个数据，打印一次已经使用的时间
    print('already use {:.3f} min'.format((time() - start_time) / 60))
    print('-----------')

    file_index += 1

file name: la_003.nii.gz
shape: (48, 160, 160)
already use 0.019 min
-----------
file name: la_004.nii.gz
shape: (48, 160, 160)
already use 0.034 min
-----------
file name: la_005.nii.gz
shape: (47, 160, 160)
already use 0.051 min
-----------
file name: la_007.nii.gz
shape: (51, 160, 160)
already use 0.069 min
-----------
file name: la_009.nii.gz
shape: (46, 160, 160)
already use 0.082 min
-----------
file name: la_010.nii.gz
shape: (47, 160, 160)
already use 0.099 min
-----------
file name: la_011.nii.gz
shape: (46, 160, 160)
already use 0.116 min
-----------
file name: la_014.nii.gz
shape: (50, 160, 160)
already use 0.132 min
-----------
file name: la_016.nii.gz
shape: (41, 160, 160)
already use 0.145 min
-----------
file name: la_017.nii.gz
shape: (45, 160, 160)
already use 0.162 min
-----------
file name: la_018.nii.gz
shape: (49, 160, 160)
already use 0.179 min
-----------
file name: la_019.nii.gz
shape: (46, 160, 160)
already use 0.193 min
-----------
file name: la_020.nii.gz
sha

In [9]:
ct_path = 'E:\skripsi\Task09_Spleen\imagesTr'
seg_path = 'E:\skripsi\Task09_Spleen\labelsTr'
upper = 350
lower = -upper
file_index = 20
counter = 0

start_time = time()

for ct_file in os.listdir(ct_path):
    if counter >=1:
        break
    
    ct = sitk.ReadImage(os.path.join(ct_path, ct_file), sitk.sitkInt16)
    ct_array = sitk.GetArrayFromImage(ct)

    seg = sitk.ReadImage(os.path.join(seg_path, ct_file), sitk.sitkInt8)
    seg_array = sitk.GetArrayFromImage(seg)

    # 对CT和金标准使用插值算法进行插值来统一轴向的spacing，插值之后的array依然是int类型
    ct_array = ndimage.zoom(ct_array, (ct.GetSpacing()[-1] / slice_thickness, down_scale, down_scale), order=3)

    # 对金标准插值不应该使用高级插值方式，这样会破坏边界部分，总之这次错误也说明了检查数据的重要性
    seg_array = ndimage.zoom(seg_array, (ct.GetSpacing()[-1] / slice_thickness, down_scale, down_scale), order=0)

    # 将灰度值在阈值之外的截断掉
    ct_array[ct_array > upper] = upper
    ct_array[ct_array < lower] = lower

    # 找到包含器官的slice
    z = np.any(seg_array, axis=(1, 2))
    print("z is ", z)
    start_slice, end_slice = np.where(z)[0][[0, -1]]
    print("start {} end {}".format(start_slice, end_slice))

    # 两个方向上各扩张slice
    if start_slice - expand_slice < 0:
        start_slice = 0
    else:
        start_slice -= expand_slice

    if end_slice + expand_slice >= seg_array.shape[0]:
        end_slice = seg_array.shape[0] - 1
    else:
        end_slice += expand_slice

    print("start {} end {}".format(start_slice, end_slice))
    ct_array = ct_array[start_slice:end_slice + 1, :, :]
    seg_array = seg_array[start_slice:end_slice + 1, :, :]

    print('file name:', ct_file)
    print('shape:', ct_array.shape)

    # 保存数据
    new_ct_array = ct_array
    new_seg_array = seg_array

    new_ct = sitk.GetImageFromArray(new_ct_array)

    new_ct.SetDirection(ct.GetDirection())
    new_ct.SetOrigin(ct.GetOrigin())
    new_ct.SetSpacing(
        (ct.GetSpacing()[0] * int(1 / down_scale), ct.GetSpacing()[1] * int(1 / down_scale), slice_thickness))

    new_seg = sitk.GetImageFromArray(new_seg_array)

    new_seg.SetDirection(ct.GetDirection())
    new_seg.SetOrigin(ct.GetOrigin())
    new_seg.SetSpacing(
        (ct.GetSpacing()[0] * int(1 / down_scale), ct.GetSpacing()[1] * int(1 / down_scale), slice_thickness))

    #new_ct_name = 'img-' + str(file_index) + '.nii'
    #new_seg_name = 'label-' + str(file_index) + '.nii'

    #sitk.WriteImage(new_ct, os.path.join(new_ct_path, new_ct_name))
    #sitk.WriteImage(new_seg, os.path.join(new_seg_path, new_seg_name))

    # 每处理完一个数据，打印一次已经使用的时间
    print('already use {:.3f} min'.format((time() - start_time) / 60))
    print('-----------')

    file_index += 1
    counter += 1

z is  [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True False False False False False
 False False False False False False False False False False False False
 False False False False False False False False]
start 38 end 66
start 28 end 76
file name: spleen_10.nii.gz
shape: (49, 256, 256)
already use 0.046 min
-----------
